In [6]:
import numpy as np
import cupy as cp
from timeit import default_timer as timer

In [7]:
mu = cp.zeros((784, 10))
sigma = cp.ones((784,10))
epochs = 1
n_mc_samples = 100
n_total_ftr = 784
lr_mu = 1
lr_sigma = 1


In [8]:
test_r = None
test_theta = None
test_eta_1 = None
test_eta_2 = None
test_eta = None
test_eta_sum = None
test_x_eta = None
test_softmax = None
test_obs_eta = None
test_derivative = None

In [9]:
def get_d(theta):
    factor = -cp.floor(cp.log10(cp.max(theta)))
    return(10**factor)


In [16]:
    def __softmax(x, y):
        global test_r 
        global test_theta
        global test_eta_1
        global test_eta_2
        global test_eta
        global test_derivative
        global test_eta_sum
        global test_x_eta
        global test_obs_eta
        global test_softmax
        """
        Update the distribution parameters mu and sigma by optimizing them in terms of the (log) likelihood.
        Here we assume a multinominal distributed target variable. We use a Multinominal model as our base model.


        :param x: (np.ndarray) Batch of observations (numeric values only, consider normalizing data for better results)
        :param y: (np.ndarray) Batch of labels: type integer e.g. 0,1,2,3,4 etc.
        """
        
        if len(x.shape) != 2:
            x = x.reshape(1,len(x))
    
        observed_classes = np.unique(y)

        for obs_class in observed_classes:
            observations_index = np.where(y == obs_class)[0]
            x_obs = cp.array(x[observations_index])
            print("x_obs: {}".format(cp.mean(cp.isinf(x_obs))))
            n_obs = len(x_obs)
            print("obs_class: {}, n obs: {}".format(obs_class, n_obs))

            for epoch in range(epochs):
                    
                    # Iterative update of mu and sigma
                    try:
                        # o number of obs, l number of samples, j features,
                        # c classes
                        
                        # r shape: oxlxjxc
                        r = cp.random.randn(n_obs, n_mc_samples,
                                            n_total_ftr,
                                            10)
                        print("r: {}".format(cp.mean(cp.isinf(r))))
                        # theta shape: oxlxjxc
                        theta = (r * sigma + mu)
                        d = 0.1
                        print(d)
                        

                        print("theta: {}".format(cp.mean(cp.isinf(theta))))
                        # eta shape: oxlxc
                        # multiply all ftr_cols with given ftr_vector x
                        eta = d * cp.einsum("oljc,oj->oljc", theta, x_obs) 
                        test_eta_1 = eta
                        # sum up all theta^cl_j * x_tj so we got l samples
                        # for all c classes
                        eta = cp.einsum("oljc->olc", eta) 
                        test_eta_2 = eta
                        eta = cp.exp(eta) # we only need them exp
                        print("eta: {}".format(cp.mean(cp.isinf(eta))))

                        # eta_sum shape: oxl
                        eta_sum = cp.einsum("olc->ol", eta)
                        print("eta_sum: {}".format(cp.mean(cp.isinf(eta_sum))))
                        # calculate softmax only for observed class
                        # obs_eta shape: oxl
                        obs_eta = eta[:,:,obs_class]
                        print("obs_eta: {}".format(cp.mean(cp.isinf(obs_eta))))
                        # softmax_lh shape: oxl
                        softmax_lh = obs_eta / eta_sum # 
                        print("softmax: {}".format(cp.mean(cp.isinf(softmax_lh))))
                        # marginal shape: o
                        marginal = cp.einsum("ol->o", softmax_lh) / \
                                   n_mc_samples
                        print("marginal: {}".format(cp.mean(cp.isinf(marginal))))

                        # calculate softmax derivative to theta
                        
                        x_eta shape: oxlxj
                        x_eta = d*cp.einsum("oj,ol->olj", x_obs, obs_eta)
                        
                        print("x_eta: {}".format(cp.mean(cp.isinf(x_eta))))
                        softmax_derivative = cp.einsum("olj,ol->olj", x_eta,
                                                       (eta_sum - obs_eta))
                        print("derivative1: {}".format(cp.mean(cp.isinf                                                     x                 (softmax_derivative))))                               
                        softmax_derivative = cp.einsum("olj->jol", softmax_derivative)
                        print("derivative2: {}".format(cp.mean(cp.isinf                                                                       (softmax_derivative))))
                        holder = softmax_derivative                     
                        print("eta_sum^2: {}".format(cp.mean(cp.isinf(eta_sum**2))))
                        softmax_derivative = cp.divide(softmax_derivative, eta_sum**2)
                        print("derivative3: {}".format(cp.mean(cp.isinf( softmax_derivative))))

                        # with new def and d

                        # softmax_derivative = d * x_obs * softmax_lh * (1 - softmax_lh)
                        #softmax_derivative = cp.einsum("oj,ol->olj", (d*x_obs),                               #                                softmax_lh)
                        #softmax_derivative = cp.einsum("olj,ol->olj",                                         #                                softmax_derivative,                                                                    (1-softmax_lh))

                        test_r = r
                        test_theta = theta
                        test_eta = eta
                        test_eta_sum = eta_sum
                        test_softmax = softmax_lh
                        #test_x_eta = x_eta
                        test_obs_eta = obs_eta
                        test_derivative = softmax_derivative
                        
                    
                        #softmax_derivative = cp.einsum("jol->olj",
                         #                              softmax_derivative)

                                                       
                        print("derivative4: {}".format(cp.mean(cp.isinf(softmax_derivative))))
                        nabla_mu = cp.einsum("olj->oj", softmax_derivative) / \
                                   n_mc_samples
                                                
                        print("nabla_mu: {}".format(cp.mean(cp.isinf(nabla_mu))))
                        r_jc = r[:,:,:,obs_class]
                        print(r_jc.shape)
                        nabla_sigma = cp.einsum("olj->oj",
                                                softmax_derivative * r_jc) / \
                                      n_mc_samples
                        print("nabla_mu: {}".format(cp.mean(cp.isinf(nabla_sigma))))

                        mu[:,obs_class] += lr_mu * \
                                                cp.einsum("jo->j",
                                                          (nabla_mu.T / marginal))
                        print("mu: {}".format(cp.mean(cp.isinf(mu))))
                        sigma[:,obs_class] += lr_sigma * \
                                                   cp.einsum("jo->j",
                                                             (nabla_sigma.T / marginal))
                        print("sigma: {}".format(cp.mean(cp.isinf(sigma))))
                        
                    
                    except TypeError as e:
                            raise TypeError('All features must be a numeric data type.') from e

SyntaxError: invalid syntax (<ipython-input-16-05c7cf909b12>, line 80)

In [3]:
from skmultiflow.data import FileStream
stream = FileStream('../datasets/Multiclass/mnist_train_normalized.csv', target_idx=0)
stream.prepare_for_use()

In [17]:
x,y = stream.next_sample(batch_size=100)

start = timer()
__softmax(x,y)
print(timer()-start)
mu[mu!=0]

x_obs: 0.0
obs_class: 0, n obs: 5
r: 0.0
0.1
theta: 0.0
eta: 0.0
eta_sum: 0.0
obs_eta: 0.0
softmax: 0.0
marginal: 0.0
derivative4: 0.0
nabla_mu: 0.0
(5, 100, 784)
nabla_mu: 0.0
mu: 0.0
sigma: 0.0
x_obs: 0.0
obs_class: 1, n obs: 13
r: 0.0
0.1
theta: 0.0
eta: 0.0
eta_sum: 0.0
obs_eta: 0.0
softmax: 0.0
marginal: 0.0
derivative4: 0.0
nabla_mu: 0.0
(13, 100, 784)
nabla_mu: 0.0
mu: 0.0
sigma: 0.0
x_obs: 0.0
obs_class: 2, n obs: 15
r: 0.0
0.1
theta: 0.0
eta: 0.0
eta_sum: 0.0
obs_eta: 0.0
softmax: 0.0
marginal: 0.0
derivative4: 0.0
nabla_mu: 0.0
(15, 100, 784)
nabla_mu: 0.0
mu: 0.0
sigma: 0.0
x_obs: 0.0
obs_class: 3, n obs: 7
r: 0.0
0.1
theta: 0.0
eta: 0.0
eta_sum: 0.0
obs_eta: 0.0
softmax: 0.0
marginal: 0.0
derivative4: 0.0
nabla_mu: 0.0
(7, 100, 784)
nabla_mu: 0.0
mu: 0.0
sigma: 0.0
x_obs: 0.0
obs_class: 4, n obs: 12
r: 0.0
0.1
theta: 0.0
eta: 0.0
eta_sum: 0.0
obs_eta: 0.0
softmax: 0.0
marginal: 0.0
derivative4: 0.0
nabla_mu: 0.0
(12, 100, 784)
nabla_mu: 0.0
mu: 0.0
sigma: 0.0
x_obs: 0.0
obs

array([0.03565111, 0.08775658, 0.08627545, ..., 0.00548781, 0.087413  ,
       0.04311852])

In [86]:

stream.restart()


In [58]:
def infs(arr):
    return(cp.mean(cp.isinf(arr)))

def nans(arr):
    return(cp.mean(cp.isnan(arr)))

In [80]:
arr = np.array([[0,1,0,0,1],[1,0,1,0,0],[1,0,0,0,0],[1,0,1,1,1]])



In [63]:
arr[0]

array([0, 1, 0, 1, 1])

In [51]:
def v (arr):
    k = np.sum(arr)
    d = len(arr)
    return np.sqrt((k/d)*(1-k/d))

In [54]:
pearson_stability(arr, arr2)

0
3
4
5
0.4898979485566356
0.39999999999999997


-2.4494897427831783

In [69]:
def pearson_stability_ij(arr1,arr2):
    d = len(arr1)
    k_i = np.sum(arr1)
    k_j = np.sum(arr2)
    x_hat_i = k_i / d
    x_hat_j = k_j / d
    arr1 = arr1 - x_hat_i
    arr2 = arr2 - x_hat_j
    dividend = 1/d * np.sum(arr1*arr2)
    divisor = np.sqrt(1/d*np.sum(arr1**2))*np.sqrt(1/d*np.sum(arr2**2))
    return dividend/divisor

In [82]:
def stability_factor(selected_ftrs):
    M = len(selected_ftrs)
    sum_stabilities = 0
    for i in range(M):
        print(i)
        for j in range(i+1, M):
            print("{}{}".format(i,j))
            print(pearson_stability_ij(selected_ftrs[i], selected_ftrs[j]))
            sum_stabilities += pearson_stability_ij(selected_ftrs[i], selected_ftrs[j])
    return 1/(M*(M-1))*sum_stabilities * 2

In [83]:
stability_factor(arr)

0
01
-0.6666666666666665
02
-0.40824829046386296
03
-0.6123724356957942
1
12
0.6123724356957945
13
0.408248290463863
2
23
0.25
3


-0.06944444444444436

In [59]:
for i in range(4,4):
    print(i)

In [ ]:
range(2,5)